In [ ]:
!pip install fastapi uvicorn nest_asyncio pyngrok

import nest_asyncio
from pyngrok import ngrok, process, conf
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import uvicorn
import asyncio
import os
import time

# Kill any existing ngrok processes using a shell command
!pkill -f ngrok || echo "No existing ngrok processes"
time.sleep(5) # Add a small delay to ensure the process terminates

# Allow nested event loops (required in Colab)
nest_asyncio.apply()

# Create FastAPI app
app = FastAPI(title="FastAPI Chat App (Colab-Safe)")

chat_history = []

class Message(BaseModel):
    sender: str
    text: str

@app.post("/chat")
def send_message(message: Message):
    chat_history.append(message.dict())
    return {"status": "Message received", "message": message}

@app.get("/history", response_model=List[Message])
def get_history():
    return chat_history

# Kill previous tunnels (extra safety)
ngrok.kill()

# Configure pyngrok with your authtoken
conf.get_default().auth_token = "33fSFHwZWMTLsBmwzMitKb2HRqO_5vLvpC7JRQRbcejRFwnUH"


# Start ngrok tunnel
port = 8000
public_url = ngrok.connect(port).public_url
print(f"Public URL: {public_url}\nDocs: {public_url}/docs")

# Configure uvicorn without asyncio.run()
config = uvicorn.Config(app=app, host="0.0.0.0", port=port, log_level="info")
server = uvicorn.Server(config)

# Run inside the existing event loop
loop = asyncio.get_event_loop()
loop.create_task(server.serve())

^C
Public URL: https://unemotional-showiest-benito.ngrok-free.dev
Docs: https://unemotional-showiest-benito.ngrok-free.dev/docs


<Task pending name='Task-1' coro=<Server.serve() running at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69>>